In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, SVR # SVM-C분류용, SVM-R회귀용
import mglearn
from sklearn.model_selection import train_test_split 

Adult/Wine 데이터셋 사용
분류 정확도 95%이상
-테스트셋이 따로 분리안되어있으면 25%를 테스트셋으로 사용
-Adult:2진분류 - 연소득 5천만 이상/이하
- wine 원산지 1,2,3 분류

- 테스트 분류 정확도 최대한 높은 모델을 찾으세요
- 무엇이 수입에 영향을 끼치는가?
  - tree, 상관관계, 계수(coefficient)

In [167]:
df = pd.read_csv('adult.data', header=None)
df.shape

(32561, 15)

In [168]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
0     32561 non-null int64
1     32561 non-null object
2     32561 non-null int64
3     32561 non-null object
4     32561 non-null int64
5     32561 non-null object
6     32561 non-null object
7     32561 non-null object
8     32561 non-null object
9     32561 non-null object
10    32561 non-null int64
11    32561 non-null int64
12    32561 non-null int64
13    32561 non-null object
14    32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [170]:
df2 = pd.read_csv('adult.test',header=0)
df2.shape

(16280, 15)

In [171]:
df2.head(2)

,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0.1,40,United-States,<=50K.
0,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
1,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.


In [172]:
df.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
df2.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']

In [173]:
dft = pd.concat((df,df2), axis=0).dropna()
dft.head(), dft.shape

(   age          workclass  fnlwgt   education  education-num  \
 0   39          State-gov   77516   Bachelors             13   
 1   50   Self-emp-not-inc   83311   Bachelors             13   
 2   38            Private  215646     HS-grad              9   
 3   53            Private  234721        11th              7   
 4   28            Private  338409   Bachelors             13   
 
         marital-status          occupation    relationship    race      sex  \
 0        Never-married        Adm-clerical   Not-in-family   White     Male   
 1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
 2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
 3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
 4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   
 
    capital-gain  capital-loss  hours-per-week  native-country  income  
 0          2174             0              40   Unit

In [174]:
dft.iloc[:,-1].unique()
#df.get_dummies(df)

array([' <=50K', ' >50K', ' <=50K.', ' >50K.'], dtype=object)

In [175]:
dft = dft.replace(' <=50K.',' <=50K')
dft = dft.replace(' >50K.',' >50K')

In [176]:
dft.iloc[:,-1].unique()

array([' <=50K', ' >50K'], dtype=object)

In [177]:
dftt = pd.get_dummies(dft)

In [178]:
dftt.shape

(48841, 110)

In [179]:
dftt.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov',
       'workclass_ Local-gov', 'workclass_ Never-worked',
       ...
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia',
       'income_ <=50K', 'income_ >50K'],
      dtype='object', length=110)

In [180]:
dftt.iloc[-5:,-1]

16275    0
16276    0
16277    0
16278    0
16279    1
Name: income_ >50K, dtype: uint8

In [181]:
y = dftt['income_ >50K']
X = dftt.iloc[:, :-2]
X.shape, y.shape

((48841, 108), (48841,))

In [182]:
#정규화 df = (df - df.min()) / (df.max() - df.min())
X_norm = (X-np.min(X,axis=0))/(np.max(X,axis=0)-np.min(X,axis=0))
X= X_norm

In [183]:
lenth = len(df)
X_train = X[:lenth]
X_test = X[lenth:]
y_train = y[:lenth]
y_test = y[lenth:]
X_train.shape, X_test.shape

((32561, 108), (16280, 108))

In [184]:
model = DecisionTreeClassifier(max_depth=5).fit(X_train,y_train)
model.score(X_test, y_test)

0.852027027027027

In [185]:
rf = RandomForestClassifier(n_estimators=100).fit(X_train,y_train)
rf.score(X_train,y_train), rf.score(X_test,y_test)

(0.9998771536500721, 0.8513513513513513)

In [186]:
import graphviz

In [187]:
import os
os.environ['PATH'] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'
os.environ['PATH']

'C:\\Users\\wtime\\Anaconda3;C:\\Users\\wtime\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\wtime\\Anaconda3\\Library\\usr\\bin;C:\\Users\\wtime\\Anaconda3\\Library\\bin;C:\\Users\\wtime\\Anaconda3\\Scripts;C:\\Users\\wtime\\Anaconda3\\bin;C:\\Users\\wtime\\Anaconda3\\condabin;C:\\Users\\wtime\\Anaconda3;C:\\Users\\wtime\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\wtime\\Anaconda3\\Library\\usr\\bin;C:\\Users\\wtime\\Anaconda3\\Library\\bin;C:\\Users\\wtime\\Anaconda3\\Scripts;C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\;C:\\Windows\\System32\\OpenSSH\\;C:\\Program Files\\Git\\cmd;C:\\Users\\wtime\\AppData\\Local\\Microsoft\\WindowsApps;;C:\\Program Files (x86)\\Graphviz2.38\\bin\\;C:\\Program Files (x86)\\Graphviz2.38\\bin\\;C:\\Program Files (x86)\\Graphviz2.38\\bin\\;C:\\Program Files (x86)\\Graphviz2.38\\bin\\;C:\\Program Files (x86)\\Graphviz2.38\\bin

In [190]:
X_train.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov',
       'workclass_ Local-gov', 'workclass_ Never-worked',
       ...
       'native-country_ Portugal', 'native-country_ Puerto-Rico',
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia'],
      dtype='object', length=108)

In [191]:
export_graphviz(model, out_file='k.dot', class_names=['0','1'], feature_names=X_train.columns, impurity=False, filled=True)
with open('k.dot') as f:
    dot_graph = f.read()
g = graphviz.Source(dot_graph)
display(g)


NameError: name 'export_graphviz' is not defined